# 词向量
- 学习词向量的概念
- 使用 skip-thought 模型训练词向量
- 使用 pytorch dataset 和 dataloader
- 学习定义 pytorch 模型
- 学习torch。nn 中的 Module
  -  Embedding
- 学习常见的 pytorch 的 operations
  - bmm
  - logsigmoid
- 保存和读取 pytorch 模型

第二课使用的训练数据可以从以下链接下载到。

链接:https://pan.baidu.com/s/1tFeK3mXuVXEy3EMarfeWvg 密码:v2z5

在这一份notebook中，我们会（尽可能）尝试复现论文Distributed Representations of Words and Phrases and their Compositionality中训练词向量的方法. 我们会实现Skip-gram模型，并且使用论文中noice contrastive sampling的目标函数。

这篇论文有很多模型实现的细节，这些细节对于词向量的好坏至关重要。我们虽然无法完全复现论文中的实验结果，主要是由于计算资源等各种细节原因，但是我们还是可以大致展示如何训练词向量。

以下是一些我们没有实现的细节

subsampling：参考论文section 2.3

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud

from collections import Counter
import numpy as np
import random
import math
import pandas as pd
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity # 余弦相似度

USE_CUDA = torch.cuda.is_available()

#为了方式 随机数对数据结果产生的影响，这里设置所有的 random.seed 是 1

np.random.seed(1)
random.seed(1)
torch.manual_seed(1)

if USE_CUDA :
    torch.cuda.manual_seed(1)
    

# set some hyper parameters
word_window = 3 # context window
samples_size = 100 # number of negative samples
NUM_EPOCH = 3
BATCH_SIZE = 128
LEARNING_RATE = 0.2
EMBEDDING_DECENT = 150
VOCAB_SIZE = 30000 # the size of vocab ,not all word in the vocab

def word_tokenize(text):
    ''' split the sequence as words '''
    return text.split()


- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [8]:

with open('text8.train.txt','r')as file:
    text = file.read()
    #text = text.split()
text = [w for w in word_tokenize(text)]

''' 使用最常见的单词制作词典实际上是不合适的，因为文本未经过处理，一些意义不大的常用词可能需要剔除 ，这一步需要事先对 text 进行处理 '''
vocab = dict (Counter(text).most_common(VOCAB_SIZE-1)) # 这里 -1 是为了给不常用词留一个位置 
vocab['<UNK>'] = len(text) - np.sum(list(vocab.values())) # vocab.values() 获取所有词的出现的次数 
vocab
id2word = [w for w in vocab.keys()]
word2id = {w: i for i,w in enumerate(id2word)}


In [9]:
word_counts =  np.array([count for count in vocab.values()],dtype = np.float32)
word_freq = word_counts / np.sum(word_counts)
# 对词频进行 3/4 次方处理
word_freq = word_freq ** (0.75)
word_freq = word_counts / np.sum(word_counts)
print(len(id2word))

30000


### 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字，然后用subsampling预处理这些文字。
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。